# tsTools Online: MEaSUREs Training Data Collection

In [1]:
from ipywidgets import Button, HBox, VBox, Label, BoundedIntText, HTML, Dropdown, Layout
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipywidgets as widgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import ipyleaflet
import geopandas
import bqplot.pyplot as plt
from dateutil import parser
import qgrid

from bqplot.interacts import BrushSelector, BrushIntervalSelector

import sqlite3
import os

from bqplot.interacts import BrushSelector, BrushIntervalSelector

# Google sheets API
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import ccd

# Turn off pandas warning
pd.options.mode.chained_assignment = None

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
# Authenticate to the Earth Engine servers, and initialize the ee module.
ee.Initialize()

In [4]:
# Functions to load collections, merge, filter and calculate statistics

# Filter collection by point and date
def collection_filtering(point, collection_name, year_range, doy_range):
    collection = ee.ImageCollection(collection_name)\
    .filterBounds(point)\
    .filter(ee.Filter.calendarRange(year_range[0], year_range[1], 'year'))\
    .filter(ee.Filter.dayOfYear(doy_range[0],doy_range[1]))
    return collection

# Landsat stack renamers, C1
def stack_renamer_l4_7_C1(img):
    band_list = ['B1', 'B2','B3','B4','B5','B7', 'B6','pixel_qa']
    name_list = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','THERMAL','pixel_qa']
    return ee.Image(img).select(band_list).rename(name_list)

def stack_renamer_l8_C1(img):
    band_list = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10','pixel_qa'];
    name_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL','pixel_qa'];
    return ee.Image(img).select(band_list).rename(name_list);


# Cloud masking for C1, L4-L7. Operators capitalized to
# avoid confusing with internal Python operators
def cloud_mask_l4_7_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(66)).Or(pqa.eq(130))\
    .Or(pqa.eq(68)).Or(pqa.eq(132))
    return ee.Image(img).updateMask(mask)
  
# Cloud masking for C1, L8
def cloud_mask_l8_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(322)).Or(pqa.eq(386)).Or(pqa.eq(324))\
    .Or(pqa.eq(388)).Or(pqa.eq(836)).Or(pqa.eq(900))
    return ee.Image(img).updateMask(mask)


# Calculate period statistics, seems to work but it's so ugly!
def calc_period_stats(yr_list, collection, reducer, stat_name, bands):
    def auxfnc(year):
        start = ee.Date(year)
        end = start.advance(step, 'year').advance(-1, 'day')

        return collection\
        .select(bands)\
        .filterDate(start, end)\
        .reduce(reducer)\
        .set({'system:time_start': start, 'system:time_end': end, 'statistic': stat_name})
    period_stats = yr_list.map(auxfnc)
    return period_stats

def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        dictarr.append(attr)

    return pd.DataFrame(dictarr)

def fc2dfgeo(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']
        dictarr.append(attr)

    df = geopandas.GeoDataFrame(dictarr)
    return df

def get_df(collection, coords, band):
    point = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = collection.filterBounds(point).select(band).getRegion(geometry=point, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['id', 'datetime', band]]
    return data

def get_df_full(collection, coords):
    point = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = collection.filterBounds(point).getRegion(geometry=point, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data['ord_time'] = data['datetime'].apply(datetime.date.toordinal)
    data = data[['id', 'datetime', 'ord_time', 'BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'THERMAL', 'pixel_qa']]
    return data


def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Load, filter and merge collections
def get_full_collection(coords, year_range, doy_range):
    point = ee.Geometry.Point(coords)
    l8_renamed = collection_filtering(point, 'LANDSAT/LC08/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l8_C1)
    l8_filtered1 = l8_renamed.map(cloud_mask_l8_C1)

    l7_renamed = collection_filtering(point, 'LANDSAT/LE07/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1);
    l7_filtered1 = l7_renamed.map(cloud_mask_l4_7_C1)

    l5_renamed = collection_filtering(point, 'LANDSAT/LT05/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1)
    l5_filtered1 = l5_renamed.map(cloud_mask_l4_7_C1)


    all_scenes = ee.ImageCollection((l8_filtered1.merge(l7_filtered1)).merge(l5_filtered1)).sort('system:time_start')
    return all_scenes

In [5]:
# PyCCD Functions
# Functions for running pyccd
def make_df_pyccd(collection, point):
    band_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    rename_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    info = collection.getRegion(point, 30).getInfo()
    header = info[0]
    data = np.array(info[1:])
    iTime = header.index('time')
    time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]
    time_new = [t.toordinal() for t in (time)]
    iBands = [header.index(b) for b in band_list]
    yData = data[0:,iBands].astype(np.float)
    df = pd.DataFrame(data=yData, index=list(range(len(yData[:,0]))), columns=rename_list)
    df['time'] = time_new
    return df

def update_df(df, df2):
    df = df.append(df2)
    return df


In [6]:
class interface(object):
    """A class to hold interface for defining sample attributes"""
    
    # Set up database
    dbPath = os.getcwd() + '/sample_database'
    if not os.path.isfile(dbPath):
        conn = sqlite3.connect(dbPath)
        c = conn.cursor()
        c.execute('''CREATE TABLE sample
                 (id real, lat real, lon real, year1 real, year2 real, class text, class2 text, confidence real)''')
        print('Database created')
    
    # Create widgets
    # Years slider
    years = widgets.IntRangeSlider(value=[2012, 2015], min=1990, max=2018,
            step=1, description='Years:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    # Primary class selector
    lc = widgets.Dropdown(options=['Select Class','Vegetation', 'Barren', 'Water','Developed'],
                value='Select Class', description='Select Class', disabled=False)

    
    # Secondary class information
    second_class_drop = widgets.Dropdown(options=['Secondary Class Information'],
                value='Secondary Class Information', description='Secondary:',
                disabled=False)
    
    # Third tier class information
    third_class_drop = widgets.Dropdown(
                            options=['Additional Class Information'],
                            value='Additional Class Information',
                            #rows=10,
                            description='Third:',
                            disabled=False
                        )
    
    # Display current validity of sample
    valid = widgets.Valid(value=False, description='Not Saved', readout='')

    
    # Create button for checking validity
    validate = widgets.Button(value=False, description='Validate', disabled=False,
                button_style='', icon='check')
    
    # Save the current sample, only possible when validity has been approved
    save_button = widgets.Button(value=False, description='Save', disabled=False,
                button_style='')
    
    # Load everything
    load_button = widgets.Button(value=False, description='Load', disabled=False,
                button_style='')
    
    valid_load = widgets.Valid(value=False, description='Not Loaded', readout='')

    
    # Manually input class label
    #textClass = widgets.Text(value='', placeholder='', description='Enter Class',
    #                disabled=False)
    
    # Interpreters confidence
    confidence = widgets.IntSlider(value=0, min=0, max=3, step=1, description='Confidence:',
                    disabled=False, continuous_update=False, orientation='horizontal',
                    readout=True, readout_format='d')
    
    # Notes
    notes = widgets.Textarea(value='Enter any useful or interesting information about the sample.',
                    placeholder='Enter any useful or interesting information about the sample',
                    description='Notes:', layout=widgets.Layout(width='70%'), disabled=False)
    
    # Spreadsheet information
    spreadsheet = widgets.Text(value='Google Spreadsheet Credential JSON',
                    placeholder='Google Spreadsheet Credential JSON',
                    description='Credentials:',disabled=False,
                    continuous_update=True)
    
    spreadName = widgets.Text(value='Google Spreadsheet Name',
                    placeholder='Google Spreadsheet Name', description='Name:',
                    disabled=False, continuous_update=True)
    
    # Sample path
    sampleWidget = widgets.Text(value='Path to sample feature collection',
                description='Path:',
                disabled=False, continuous_update=True)
    
    def __init__(self):
        interface.sample_path = None
        interface.sheet = None
  
    
    def lc_clicked(selection):
        """ Generate secondary class selector after initial class is chosen """
        if selection.new == 'Vegetation':
            interface.second_class_drop.set_trait('options', ['Vegetation Growth Form','Cropland', 'Tree', 'Shrub','Herbaceous'])
        elif selection.new == 'Barren':
            interface.second_class_drop.set_trait('options', ['Barren Form','Sand/Beach', 'Rock', 'Ice/Snow'])
        elif selection.new == 'Developed':
            interface.second_class_drop.set_trait('options', ['Developed Form','Low Albedo', 'High Albedo'])
        elif selection.new == 'Water':
            interface.second_class_drop.set_trait('options', ['Water Form','Shore/Intertidal', 'Open Water'])
    
    def third_clicked(selection):
        """ Generate additional class selector after initial class is chosen """
        if selection.new == 'Herbaceous':
            interface.third_class_drop.set_trait('options', ['Additional Class Information','Moss/lichen', 'Natural grassland', 'Pastures','Grass/Lawn'])
        elif selection.new == 'Tree' or selection == 'Shrub':
            interface.third_class_drop.set_trait('options', ['Additional Class Information','Deciduous','Evergreen','Mixed'])
        elif selection.new == 'Cropland':
            interface.third_class_drop.set_trait('options', ['Only secondary information needed'])
        elif interface.lc.value == 'Barren' or interface.lc.value == 'Water' or interface.lc.value == 'Developed':
             interface.third_class_drop.set_trait('options', ['Only secondary information needed'])
    
    def check_val_status(selection):
        """ Check the validity of the current sample and change valid widget accordingly """
        selected_secondary_lc = False
        wrote_correct_lc = False
        if interface.second_class_drop.value != 'Secondary Class Information':
            selected_secondary_lc = True
        else:
            print("Must specify secondary class information!")
        if interface.lc.value.capitalize() == interface.textClass.value.capitalize():
            wrote_correct_lc = True
        if selected_secondary_lc and wrote_correct_lc:
            interface.valid.value = True
            interface.save_button.disabled = False
            #interface.valid.set_trait('value',True)
            #interface.save_button.set_trait('disabled',False)
  
    
    def save_sample(selection):
        """ Save the sample to the database """
        # TODO: This should be backed up somehow
        
        # Connect to the database
        conn = sqlite3.connect(interface.dbPath)
        c = conn.cursor()
        
        # Get everything in right format
        year1 = interface.years.value[0]
        year2 = interface.years.value[1]
        lcClass = interface.lc.value
        class2 = interface.second_class_drop.value
        class3 = interface.third_class_drop.value
        conf = interface.confidence.value

        # TODO: Read ID and geographic information from sample
        idSample = Plot_interface.current_id
        lat = Plot_interface.m.center[0]
        lon = Plot_interface.m.center[1]

        sampleInput = (idSample, lat, lon, year1, year2, lcClass, class2, conf)

        # Put sample information into database
        c.execute("""insert into sample
                  values {i}""".format(i=sampleInput))

        # Save (commit) the changes
        conn.commit()

        # Close the cursor
        c.close()
        
        # Change save validity state
        interface.valid.value = True
        interface.valid.description='Saved!'

        # Reset the buttons
        interface.years.set_trait('value',[2012, 2015])
        interface.lc.set_trait('value', 'Select Class')
        
        # Save to drive
        sampleInputList = [idSample, lat, lon, year1, year2, lcClass, class2, class3, conf]
        interface.sheet.insert_row(sampleInputList, 2)

    def load_everything(sender):
        # Load Google Spreadsheet and GEE Feature Collection
        scope = ['https://spreadsheets.google.com/feeds' + ' ' +'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(interface.spreadsheet.value, scope)
        client = gspread.authorize(creds)
        interface.sheet = client.open(interface.spreadName.value).sheet1
        interface.sample_path = ee.FeatureCollection(interface.sampleWidget.value)
        interface.df = fc2dfgeo(interface.sample_path)
        interface.df = interface.df.set_index('ID')
        interface.valid_load.value = True
        interface.valid_load.description='Loaded!'
        
    # Handle widget interactions
    # Call function check_val_status when button is clicked
    validate.on_click(check_val_status)    
    # Primary class selector action
    lc.observe(lc_clicked, 'value')
    # Secondary class selector action
    second_class_drop.observe(third_clicked, 'value')
    # Call function save_sample when clicking save button
    save_button.on_click(save_sample)
    
    # Load database and sample
    load_button.on_click(load_everything)


In [7]:
# Set global variables
# year_range = [ 2000, 2018 ];
# doy_range = [ 1, 365 ];
# step = 1 #in years
#dbPath = "/home/paulo/default_bu/notebooks/database/test.db"

In [8]:
class Plot_interface(object):
    """Class to handle map and plot interaction"""

    # Declare class attributes
    current_id = 0
    pyccd_flag = False
    pyccd_flag2 = False
    current_band = ''
    band_index1 = 4
    band_index2 = 4
    click_col = ''
    click_df = pd.DataFrame()
    sample_col = ''
    sample_df = pd.DataFrame()
    PyCCDdf = pd.DataFrame()
    table = pd.DataFrame()
    band_list =['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2']
    year_range = [ 1986, 2018 ]
    doy_range = [ 1, 365 ]
    step = 1 #in years
    
    # Create widget controls
    next_pt = Button(value=False, description='Next point', disabled=False) 
    previous_pt = Button(value=False, description='Previous point', disabled=False)
    pyccd_button = Button(value=False, description='Run PyCCD 1', disabled=False)
    pyccd_button2 = Button(value=False, description='Run PyCCD 2', disabled=False)
    band_selector1 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Select band', value=None)
    band_selector2 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Select band', value=None)
    
    ylim = widgets.IntRangeSlider(value=[0, 4000], min=0, max=10000,
            step=500, description='YLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    xlim = widgets.IntRangeSlider(value=[2000, 2018], min=1986, max=2018,
            step=1, description='XLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    ylim2 = widgets.IntRangeSlider(value=[0, 4000], min=0, max=10000,
            step=500, description='YLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    xlim2 = widgets.IntRangeSlider(value=[2000, 2018], min=1986, max=2018,
            step=1, description='XLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    
    coords_label = Label()
    pt_message = HTML("Current ID: {}".format(current_id))
    time_label = HTML(value='')
    selected_label = HTML("ID of selected point")
    hover_label = HTML("test value")
    text_brush = HTML(value = 'Selected year range:')
    kml_link = HTML(value = 'KML:')
    
    # Create map including streets and satellite and controls
    m = ipyleaflet.Map(zoom=5, layout={'height':'400px'}, 
                       center=(3.3890701010382958, -67.32297252983098),dragging=True,
                       close_popup_on_click=False, basemap=ipyleaflet.basemaps.Esri.WorldStreetMap)
    
    streets = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.Esri.WorldImagery)
    m.add_layer(streets)

    dc = ipyleaflet.DrawControl(marker={'shapeOptions': {'color': '#ff0000'}}, 
                                polygon={}, circle={}, circlemarker={}, polyline={})
    
    
    # Table widget
    table_widget = qgrid.show_grid(table, show_toolbar=False)
    
    # Set plots
    # Plot scales.
    lc1_x = bqplot.DateScale(min=datetime.date(xlim.value[0], 2, 1), max=datetime.date(xlim.value[1], 1, 1))
    lc2_y = bqplot.LinearScale(min=ylim.value[0], max=ylim.value[1])
    
    lc1_x2 = bqplot.DateScale(min=datetime.date(xlim.value[0], 2, 1), max=datetime.date(xlim.value[1], 1, 1))
    lc2_y2 = bqplot.LinearScale(min=ylim.value[0], max=ylim.value[1])

    # Main scatter plot for samples
    lc2 = bqplot.Scatter(
        x=[],
        y=[],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        interactions={'click': 'select', 'hover': 'tooltip'},
        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
        unselected_style={'opacity': 0.5},
        display_legend=True,
        labels = ['Sample point']
    )
    
    # Pyccd model fit
    lc4 = bqplot.Lines(
        x=[],
        y=[],
        colors=['black'],
        stroke_width=3,
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
    )
    
    # Pyccd model break
    lc5 = bqplot.Scatter(
        x=[],
        y=[],
        marker='triangle-up',
        colors=['red'],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        display_legend=False,
        labels = ['Model Endpoint']
    )
    
    # Scatter plot for clicked points in map
    lc3 = bqplot.Scatter(
        x=[],
        y=[],
        scales={'x': lc1_x2, 'y': lc2_y2},
        size=[1,1],
        colors=['gray'],
        interactions={'click': 'select', 'hover': 'tooltip'},
        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
        unselected_style={'opacity': 0.5},
        display_legend=True,
        labels = ['Clicked point']
    )
    
    # Pyccd model fit for clicked point
    lc6 = bqplot.Lines(
        x=[],
        y=[],
        colors=['black'],
        stroke_width=3,
        scales={'x': lc1_x2, 'y': lc2_y2},
        size=[1,1],
    )
    
    # Pyccd model break for clicked point
    lc7 = bqplot.Scatter(
        x=[],
        y=[],
        marker='triangle-up',
        colors=['red'],
        scales={'x': lc1_x2, 'y': lc2_y2},
        size=[1,1],
        display_legend=False,
        labels = ['Model Endpoint']
    )

    # Plot axes.
    x_ax1 = bqplot.Axis(label='Date', scale=lc1_x, num_ticks = 6, tick_format='%Y')
    x_ax2 = bqplot.Axis(label='Date', scale=lc1_x2, num_ticks = 6, tick_format='%Y')
    x_ay1 = bqplot.Axis(label='SWIR1', scale=lc2_y, orientation='vertical')
    x_ay2 = bqplot.Axis(label='SWIR1', scale=lc2_y2, orientation='vertical')


    # Create a figure for sample points.
    fig = bqplot.Figure(
        marks=[lc2, lc4, lc5],
        axes=[x_ax1, x_ay1],
        #layout={'height':'300px', 'width':'800px'},
        layout={'height':'300px', 'width':'100%'},
        title="Sample TS"
        #title_style={'font-size': '12px'}
    )
    
    # Create a figure for clicked points.
    fig2 = bqplot.Figure(
        marks=[lc3, lc6, lc7],
        axes=[x_ax2, x_ay2],
        #layout={'height':'300px', 'width':'800px'},
        layout={'height':'300px', 'width':'100%'},
        title="Clicked TS"
        #title_style={'font-size': '12px'}
    )

    def __init__(self, navigate):
        Plot_interface.navigate = navigate
        Plot_interface.band_index1 = 4
        Plot_interface.band_index2 = 4
        Plot_interface.pyccd_flag = False
        Plot_interface.pyccd_flag2 = False
        Plot_interface.table = None  
        # Set up database
        conn = sqlite3.connect(Plot_interface.navigate.dbPath)
        Plot_interface.c = conn.cursor()
        Plot_interface.minv = 0
        Plot_interface.maxv = 6000
        Plot_interface.b1 = 'SWIR1'
        Plot_interface.b2 = 'NIR'
        Plot_interface.b3 = 'RED'
           
    @classmethod
    def map_point(self):
        gjson = ipyleaflet.GeoJSON(data=Plot_interface.navigate.df['geometry'][Plot_interface.current_id], name="Sample point")
        Plot_interface.m.center = gjson.data['coordinates'][::-1]
        Plot_interface.m.zoom = 12
        Plot_interface.m.add_layer(gjson)
        kmlstr = ee.FeatureCollection(ee.Geometry.Point(Plot_interface.navigate.df['geometry'][Plot_interface.current_id]['coordinates'])).getDownloadURL("kml")
        Plot_interface.kml_link.value = "<a '_blank' rel='noopener noreferrer' href={}>KML Link</a>".format(kmlstr)
        
    @classmethod
    def get_ts(self):
        current_band = Plot_interface.band_list[Plot_interface.band_index1]
        Plot_interface.sample_col = get_full_collection(Plot_interface.navigate.df['geometry'][Plot_interface.current_id]['coordinates'], 
                                       Plot_interface.year_range, Plot_interface.doy_range)
        Plot_interface.sample_df = get_df_full(Plot_interface.sample_col, Plot_interface.navigate.df['geometry'][Plot_interface.current_id]['coordinates']).dropna()
    
    @classmethod
    def plot_ts(self):
        current_band = Plot_interface.band_list[Plot_interface.band_index1]
        Plot_interface.lc2.x = Plot_interface.sample_df['datetime']
        Plot_interface.lc2.y = Plot_interface.sample_df[current_band]
        Plot_interface.x_ay1.label = current_band
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        # TODO: This should not have to be ran again
        #if pyccd_flag:
        if Plot_interface.pyccd_flag:
            Plot_interface.run_pyccd(0)
        
    # Go back or forth between sample points
    def advance(b):
        # Plot point in map
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        Plot_interface.lc5.display_legend=False
        Plot_interface.pyccd_flag = False
        Plot_interface.current_id += 1
        Plot_interface.pt_message.value = "Point ID: {}".format(Plot_interface.current_id)
        Plot_interface.map_point() 
        Plot_interface.get_ts()
        Plot_interface.plot_ts()
        Plot_interface.change_table()
        
    def decrease(b):
        # Plot point in map
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        Plot_interface.lc5.display_legend=False
        Plot_interface.pyccd_flag = False
        Plot_interface.current_id -= 1
        Plot_interface.pt_message.value = "Point ID: {}".format(Plot_interface.current_id)
        Plot_interface.map_point() 
        Plot_interface.get_ts()
        Plot_interface.plot_ts()    
        Plot_interface.change_table()
        
    def change_table():
        # Update the table based on current ID

        # Get header
        cursor = Plot_interface.c.execute('select * from sample')
        names = list(map(lambda x: x[0], cursor.description))
        previous_inputs = pd.DataFrame()
        for i, row in enumerate(Plot_interface.c.execute("SELECT * FROM sample WHERE id = '%s'" % Plot_interface.current_id)):
            previous_inputs[i] = row
        #print(previous_inputs)
        previous_inputs = previous_inputs.T
        if previous_inputs.shape[0] > 0:
            previous_inputs.columns = names
        Plot_interface.table_widget.df = previous_inputs
        #Plot_interface.table = previous_inputs
        #display(previous_inputs)
    
    # Band selection for sample point
    def on_band_selection1(change):        
        new_band = change['new']
        #global band_index
        band_index = change['owner'].index
        Plot_interface.band_index1 = band_index
        Plot_interface.plot_ts()

    # Band selection for clicked point
    def on_band_selection2(change):        
        new_band = change['new']
        band_index = change['owner'].index
        Plot_interface.band_index2 = band_index
        Plot_interface.lc3.x = Plot_interface.click_df['datetime']
        Plot_interface.lc3.y = Plot_interface.click_df[new_band]
        Plot_interface.x_ay2.label = new_band
        if Plot_interface.pyccd_flag2:
            Plot_interface.run_pyccd2(0)

    def change_yaxis(value):
        Plot_interface.lc2_y.min = Plot_interface.ylim.value[0]
        Plot_interface.lc2_y.max = Plot_interface.ylim.value[1]

    def change_xaxis(value):
        Plot_interface.lc1_x.min = datetime.date(Plot_interface.xlim.value[0], 2, 1)
        Plot_interface.lc1_x.max = datetime.date(Plot_interface.xlim.value[1], 2, 1)
    
    def change_yaxis2(value):
        Plot_interface.lc2_y2.min = Plot_interface.ylim2.value[0]
        Plot_interface.lc2_y2.max = Plot_interface.ylim2.value[1]

    def change_xaxis2(value):
        Plot_interface.lc1_x2.min = datetime.date(Plot_interface.xlim2.value[0], 2, 1)
        Plot_interface.lc1_x2.max = datetime.date(Plot_interface.xlim2.value[1], 2, 1)
    
    
    def hover_event(self, target):
        Plot_interface.hover_label.value = str(target['data']['x'])
    
    # Add layer from clicked point in sample TS figure
    def click_event(self, target):
        pt_index = target['data']['index']
        current_band = Plot_interface.band_list[Plot_interface.band_index1]
        image_id = Plot_interface.sample_df['id'].values[pt_index]
        selected_image = ee.Image(Plot_interface.sample_col.filterMetadata('system:index', 'equals', image_id).first())
        tile_url = GetTileLayerUrl(selected_image.visualize(min=Plot_interface.minv, 
                                                            max=Plot_interface.maxv, 
                                                            bands= [Plot_interface.b1, Plot_interface.b2, 
                                                                    Plot_interface.b3]))

        Plot_interface.m.add_layer(ipyleaflet.TileLayer(url=tile_url, name=image_id))

    # Add layer from clicked point in clicked TS figure
    def click_event2(self, target):
        pt_index = target['data']['index']
        current_band = Plot_interface.band_list[Plot_interface.band_index2]
        #Find clicked image. .values needed to access the nth element of that list instead of indexing by ID
        image_id = Plot_interface.click_df['id'].values[pt_index]
        selected_image = ee.Image(Plot_interface.click_col.filterMetadata('system:index', 'equals', image_id).first())
        tile_url = GetTileLayerUrl(selected_image.visualize(min=Plot_interface.minv, 
                                                            max=Plot_interface.maxv, 
                                                            bands= [Plot_interface.b1, Plot_interface.b2, 
                                                                    Plot_interface.b3]))

        Plot_interface.m.add_layer(ipyleaflet.TileLayer(url=tile_url, name=image_id))
    
    # Plot TS from clicked point
    def handle_draw(self, action, geo_json):
        # Get the selected coordinates from the map's drawing control.
        current_band = Plot_interface.band_list[Plot_interface.band_index2]
        coords = geo_json['geometry']['coordinates']    
        Plot_interface.click_col = get_full_collection(coords, Plot_interface.year_range, Plot_interface.doy_range)
        Plot_interface.click_df = get_df_full(Plot_interface.click_col, coords).dropna()
        Plot_interface.lc6.x = []
        Plot_interface.lc6.y = []
        Plot_interface.lc7.x = []
        Plot_interface.lc7.y = []
        Plot_interface.lc3.x = Plot_interface.click_df['datetime']
        Plot_interface.lc3.y = Plot_interface.click_df[current_band]
    
    # Plotting pyccd
    def plot_pyccd(results, band, plotband, dates, yl, ylabel, ts_type):
        mask = np.array(results['processing_mask']).astype(np.bool_)
        predicted_values = []
        prediction_dates = []
        break_dates = []
        start_dates = []

        for num, result in enumerate(results['change_models']):
            days = np.arange(result['start_day'], result['end_day'] + 1)
            prediction_dates.append(days)
            break_dates.append(result['break_day'])
            start_dates.append(result['start_day'])
            intercept = result[list(result.keys())[6+band]]['intercept']
            coef = result[list(result.keys())[6+band]]['coefficients']

            predicted_values.append(intercept + coef[0] * days +
                                    coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
                                    coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
                                    coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))

        num_breaks = len(break_dates)

        # eric here
        break_y = [plotband[dates == i][0] for i in break_dates]

        #break_y = [0] * num_breaks
        break_dates_plot = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in break_dates]

        plot_dates = np.array([datetime.datetime.fromordinal(i) for i in (dates)])

        # Predicted curves
        all_dates = []
        all_preds = []
        for _preddate, _predvalue in zip(prediction_dates, predicted_values):
            all_dates.append(_preddate)
            all_preds.append(_predvalue)

        all_preds = [item for sublist in all_preds for item in sublist]
        all_dates = [item for sublist in all_dates for item in sublist]

        date_ord = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in all_dates]
        _x = np.array(date_ord, dtype='datetime64')
        _y = all_preds
        
        if ts_type == 'sample_ts':
            Plot_interface.lc4.x = _x
            Plot_interface.lc4.y = _y
            Plot_interface.lc5.x = np.array(break_dates_plot, dtype='datetime64')
            Plot_interface.lc5.y = break_y
        elif ts_type == 'clicked_ts':
            Plot_interface.lc6.x = _x
            Plot_interface.lc6.y = _y
            Plot_interface.lc7.x = np.array(break_dates_plot, dtype='datetime64')
            Plot_interface.lc7.y = break_y
        

    # Run pyccd
    def run_pyccd(b):
        # Run pyCCD on current point
        Plot_interface.pyccd_flag = True
        Plot_interface.lc5.display_legend=True
        dfPyCCD = Plot_interface.sample_df
    

#         dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'].isnull()] = 4
#         dfPyCCD[df.isnull()] = 0
        dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'] > 4] = 0

        #TODO: Paramaterize everything
        params = {'QA_BITPACKED': False,
                  'QA_FILL': 255,
                  'QA_CLEAR': 0,
                  'QA_WATER': 1,
                  'QA_SHADOW': 2,
                  'QA_SNOW': 3,
                  'QA_CLOUD': 4}

        dates = np.array(dfPyCCD['ord_time'])
        blues = np.array(dfPyCCD['BLUE'])
        greens = np.array(dfPyCCD['GREEN'])
        reds = np.array(dfPyCCD['RED'])
        nirs = np.array(dfPyCCD['NIR'])
        swir1s = np.array(dfPyCCD['SWIR1'])
        swir2s = np.array(dfPyCCD['SWIR2'])
        thermals = np.array(dfPyCCD['THERMAL'])
        qas = np.array(dfPyCCD['pixel_qa'])
        results = ccd.detect(dates, blues, greens, reds, nirs, swir1s, swir2s, thermals, qas, params=params)

        band_names = ['Blue SR', 'Green SR', 'Red SR', 'NIR SR', 'SWIR1 SR', 'SWIR2 SR','THERMAL']
        plotlabel = band_names[Plot_interface.band_index1] 

        plot_arrays = [blues, greens, reds, nirs, swir1s, swir2s]
        plotband = plot_arrays[Plot_interface.band_index1]
        Plot_interface.plot_pyccd(results, Plot_interface.band_index1, 
                                  plotband, dates, (0, 4000), 'PyCCD Results', 'sample_ts')
    
    def run_pyccd2(b):
        # Run pyCCD on current point
        Plot_interface.pyccd_flag2 = True
        
        # Display the legend
        Plot_interface.lc7.display_legend=True
        
        dfPyCCD = Plot_interface.click_df

        # First two lines no longer required bc we are removing NA's when we load the TS
        #dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'].isnull()] = 4
        #dfPyCCD[df.isnull()] = 0
        dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'] > 4] = 0

        #TODO: Paramaterize everything
        params = {'QA_BITPACKED': False,
                  'QA_FILL': 255,
                  'QA_CLEAR': 0,
                  'QA_WATER': 1,
                  'QA_SHADOW': 2,
                  'QA_SNOW': 3,
                  'QA_CLOUD': 4}

        dates = np.array(dfPyCCD['ord_time'])
        blues = np.array(dfPyCCD['BLUE'])
        greens = np.array(dfPyCCD['GREEN'])
        reds = np.array(dfPyCCD['RED'])
        nirs = np.array(dfPyCCD['NIR'])
        swir1s = np.array(dfPyCCD['SWIR1'])
        swir2s = np.array(dfPyCCD['SWIR2'])
        thermals = np.array(dfPyCCD['THERMAL'])
        qas = np.array(dfPyCCD['pixel_qa'])
        results = ccd.detect(dates, blues, greens, reds, nirs, swir1s, swir2s, thermals, qas, params=params)

        band_names = ['Blue SR', 'Green SR', 'Red SR', 'NIR SR', 'SWIR1 SR', 'SWIR2 SR','THERMAL']
        plotlabel = band_names[Plot_interface.band_index2] 

        plot_arrays = [blues, greens, reds, nirs, swir1s, swir2s]
        plotband = plot_arrays[Plot_interface.band_index2]
        Plot_interface.plot_pyccd(results, Plot_interface.band_index2, 
                                  plotband, dates, (0, 4000), 'PyCCD Results', 'clicked_ts')
    
    ylim.observe(change_yaxis)
    xlim.observe(change_xaxis)
    ylim2.observe(change_yaxis2)
    xlim2.observe(change_xaxis2)
    next_pt.on_click(advance)
    previous_pt.on_click(decrease)
    pyccd_button.on_click(run_pyccd)
    pyccd_button2.on_click(run_pyccd2)
    band_selector1.observe(on_band_selection1, names='value')
    band_selector2.observe(on_band_selection2, names='value')
    
    lc2.on_element_click(click_event)
    lc2.tooltip=hover_label
    lc2.on_hover(hover_event)

    lc3.on_element_click(click_event2)
    lc3.tooltip=hover_label
    lc3.on_hover(hover_event)
    
    dc.on_draw(handle_draw)
    m.add_control(dc)
    m.add_control(ipyleaflet.LayersControl())

In [9]:
display(HTML('<style>.container { width:100% !important; }</style>'))

HTML(value='<style>.container { width:100% !important; }</style>')

In [10]:
newInterface = interface()
newPlot = Plot_interface(newInterface)

navigate_samples = HBox(children=[newPlot.previous_pt, newPlot.next_pt])
load_data = HBox(children=[newInterface.load_button, newInterface.valid_load])
tab0 = VBox(children=[newInterface.spreadsheet, newInterface.spreadName, newInterface.sampleWidget,
                      load_data])

tab1 = VBox(children=[newInterface.years,
                      newInterface.lc,
                      newInterface.second_class_drop,
                      newInterface.third_class_drop,
                      newInterface.notes
                      ])
sample_pyccd = HBox([newPlot.pyccd_button, newPlot.pt_message, newPlot.kml_link])
sample_ts_box = VBox([HTML(value="<b>SAMPLE TS</b>"), newPlot.band_selector1, newPlot.ylim, newPlot.xlim, navigate_samples, sample_pyccd])
clicked_ts_box = VBox([HTML(value="<b>CLICKED TS</b>"), newPlot.band_selector2, newPlot.ylim2, newPlot.xlim2, newPlot.pyccd_button2])
tab2 = widgets.Tab([HBox([sample_ts_box, clicked_ts_box])])
tab2.set_title(0, 'Controls')
tab3 = VBox(children=[newInterface.confidence,
                      newInterface.save_button,
                      newInterface.valid])
tab4 = VBox(children=[newPlot.table_widget])

tab = widgets.Tab(children=[tab0, tab1, tab3, tab4], layout=Layout(width='100%', min_height='270px', max_height='270px'))
tab.set_title(0, 'Load sample')
tab.set_title(1, 'Classification')
tab.set_title(2, 'Interpretation')
tab.set_title(3, 'Table')

display(newPlot.m)
fig = HBox(children=[newPlot.fig, newPlot.fig2])
controls = HBox([tab2, tab])
tabfig = VBox([controls, fig])
display(tabfig)

#/home/paulo/tstools_online/notebooks/client_secret.json
#test_sample
#users/parevalo_bu/sample_13_15_labels_pts

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{…